In [378]:
import pandas as pd
import bs4
from bs4 import BeautifulSoup
import re
import datetime

contents = open('transcript.html').read()

soup = BeautifulSoup(contents, 'html')

In [146]:
sectionHeaders = soup.findAll('span', {"class": 'fieldOrangetextbold'})
quartersBlock = sectionHeaders[1:]

In [393]:
def parseRecord(element):
    return [i.text.strip() for i in element.children if type(i) != bs4.element.NavigableString]

def quarterToDate(quarter):
    matches = re.match(r'([a-zA-z]+) Quarter ([0-9]+)-([0-9]+)', quarter)
    season, _year, _ = matches.groups()
    seasonMonth = {'Fall': 9, 'Winter': 1, 'Spring': 4, 'Summer': 7}
    yearAdjustment = 1 if season in ['Winter', 'Spring', 'Summer'] else 0
    year = int('20' + str(int(_year) + yearAdjustment))
    return datetime.date(year=year, month=seasonMonth[season], day=1)

In [404]:
def getDataFromSectionHeader(header):
    headerRow = header.parent.parent
    curRow = headerRow
    for i in range(16):
        curRow = curRow.nextSibling
        if type(curRow) == bs4.element.NavigableString:
            continue
        firstColContents = [child for child in curRow.children][1].text
        if firstColContents == 'Subject':
            break
    cols = [col.text for col in curRow.children if type(col) != bs4.element.NavigableString]
    curData = curRow.nextSibling.nextSibling
    records = []
    while curData is not None and not re.search('Term Totals', curData.text):
        records.append(parseRecord(curData))
        curData = curData.nextSibling
        if type(curData) == bs4.element.NavigableString:
            curData = curData.nextSibling
    excluded = ['Start and End Dates', 'Quality Points', 'Level', 'R']
    data = [dict(zip(cols, record)) for record in records]
    quarter = headerRow.text.strip().replace("Term: ", '')
    for record in data:
        for col in excluded:
            if col in record:
                del record[col]
        record['Quarter'] = quarter
        record['Date'] = quarterToDate(quarter)
    return data

In [405]:
nestedData = [getDataFromSectionHeader(_header) for _header in quartersBlock]
flatData = []
# leave out last quarter -- i ain't done yet!!! lol fuck
for quarter in nestedData[:-1]:
    for record in quarter:
        flatData.append(record)
transcript = pd.DataFrame(flatData)

transcript

,Course,Credit Hours,Date,Grade,Quarter,Subject,Title
0,121,1.000,2011-09-01,A,Fall Quarter 11-12,CS,Computation Lab I
1,164,3.000,2011-09-01,A+,Fall Quarter 11-12,CS,Intro to Computer Science
2,101,3.000,2011-09-01,A+,Fall Quarter 11-12,ENGL,Expository Writing and Reading
3,101,2.000,2011-09-01,A,Fall Quarter 11-12,ENGR,Engineering Design Lab I
4,200,1.000,2011-09-01,A,Fall Quarter 11-12,HNRS,Introduction to Honors Program
5,121,4.000,2011-09-01,A,Fall Quarter 11-12,MATH,Calculus I
6,101,3.000,2011-09-01,A-,Fall Quarter 11-12,PHIL,Intro to Western Philosophy
7,E101,1.000,2011-09-01,A,Fall Quarter 11-12,UNIV,The Drexel Experience
8,101,0.000,2012-01-01,CR,Winter Quarter 11-12,COOP,Career Mgmt/Profess Dev
9,122,1.000,2012-01-01,A+,Winter Quarter 11-12,CS,Computation Lab II
